# Рецепты

# Data

In [8]:
from glob import glob


def make_translation(source, target):
    assert len(source) == len(target)
    return {
        ord(a): ord(b)
        for a, b in zip(source, target)
    }


DASHES_TRANSLATION = make_translation(
    '‑–—−',
    '----'
)


def preprocess(text):
    text = text.replace('\xa0', ' ')
    text = text.replace('\xad', '')
    text = text.translate(DASHES_TRANSLATION)
    return text


texts = []
for path in sorted(glob('texts/*.txt')):
    with open(path) as file:
        text = file.read()
        text = preprocess(text)
        texts.append(text)

In [9]:
from random import seed, sample

seed(40)
for text in sample(texts, 2):
    print(text)
    print('---' * 10)

Мексиканский суп с курицей

Нам потребуется:

Куриная грудка - 1 шт.,
Лук репчатый - 1 шт.,
Красная фасоль консервированная - 1 банка,
Вермишель - 50 г,
Кукуруза замороженная - 100 г,
Укроп - по вкусу,
Соль - по вкусу,
Молотый острый красный перец - по вкусу.

Из куриной грудки сварите бульон, достаньте мясо, отделите от костей и мелко нарежьте.

Лук мелко нашинкуйте. В глубоком сотейнике разогрейте растительное масло и обжарьте лук до прозрачности.

Добавьте к луку курицу и обжаривайте несколько минут. С фасоли слейте жидкость. Переложите фасоль в сотейник, добавьте молотый перец по вкусу, перемешайте и продолжайте обжаривать.

Залейте в сотейник куриный бульон, посолите по вкусу и варите 5 мин. Добавьте вермишель и кукурузу, варите еще 5 минут, затем накройте крышкой и снимите с огня.

Дайте супу настояться. Мелко нарубите укроп. Подавайте суп с укропом и кукурузными чипсами.
------------------------------
Мясной пирог

Ингредиенты:
для теста:
сливочное масло - 1/2 стак
сметана - 1 с

In [10]:
import re


def find_ingredient_sections(text):
    return re.findall(r'ингредиенты:(.+?)приготовление', text, re.I | re.S)


def maybe_ingredient(line):
    match = re.search(r'\d', line)
    size = len(line) <= 50
    return match and size


lines = []
for text in texts:
    sections = find_ingredient_sections(text)
    for section in sections:
        for line in section.splitlines():
            if maybe_ingredient(line):
                lines.append(line)

In [11]:
seed(14)
sample(lines, 20)

['● Оливковое масло - 2 ст. л.',
 'Молоко 1% - 100 г ',
 '- Морковь 2 - 3 шт.',
 'творог - 500 г,',
 '2 яйца,',
 '* 3/4 стакана холодной кипяченой воды',
 '• сыр - 100 г.',
 '1 нарезанный красный перец',
 '230 г кокосового или тростникового сахара ',
 'Яйцо - 3 шт.',
 'Сок лайма - 2 Ст. ложки',
 '● Мука - 100 г.',
 'Сметана - 50 г',
 '● Бананы - 3 шт.',
 'ананасы консервированные - 1 банка ',
 '● Сыр твердый 30 гр.',
 '500 мл растительного масла ',
 '5) соль, растительное масло для пассеровки овощей.',
 '5 средних варёных картофелин ',
 '● 3-5 зубчиков чеснока; ']

# Grammar

In [12]:
from IPython.display import display

# pip install ipymarkup
from ipymarkup import show_span_ascii_markup as show_markup

from yargy import (
    Parser,
    or_, rule
)
from yargy.pipelines import morph_pipeline
from yargy.predicates import (
    eq, in_, dictionary,
    type, gram
)
from yargy.tokenizer import MorphTokenizer
from yargy import interpretation as interp
from yargy.interpretation import fact, attribute


def show_matches(rule, *lines):
    parser = Parser(rule)
    for line in lines:
        matches = parser.findall(line)
        matches = sorted(matches, key=lambda _: _.span)
        spans = [_.span for _ in matches]
        show_markup(line, spans)
        if matches:
            facts = [_.fact for _ in matches]
            if len(facts) == 1:
                facts = facts[0]
            display(facts)


INT = type('INT')
NOUN = gram('NOUN')
ADJF = gram('ADJF')
PRTF = gram('PRTF')
GENT = gram('gent')
DOT = eq('.')

TOKENIZER = MorphTokenizer()

## Unit

### Spoon

In [13]:
Spoon = fact(
    'Spoon',
    ['type']
)


TEA = morph_pipeline([
    'ч',
    'чайн',
    'чайная',
]).interpretation(
    Spoon.type.const('чайная')
)

TABLE = morph_pipeline([
    'ст',
    'стол',
    'столовая',
]).interpretation(
    Spoon.type.const('столовая')
)

NAME = morph_pipeline([
    'л',
    'ложка'
])

SPOON = rule(
    or_(TEA, TABLE),
    DOT.optional(),
    NAME,
    DOT.optional()
).interpretation(
    Spoon
)


show_matches(
    SPOON,
    '2 Ст. ложки',
    '3 чайных л',
)

2 Ст. ложки
  ─────────


Spoon(
    type='столовая'
)

3 чайных л
  ────────


Spoon(
    type='чайная'
)

### Unit

In [15]:
from yargy.record import Record


Unit = fact(
    'Unit',
    ['type', 'modifier']
)


class Synonyms(Record):
    __attributes__ = ['name', 'synonyms']
    
    def __init__(self, name, synonyms=()):
        self.name = name
        self.synonyms = synonyms


NAMES = [
    Synonyms('банка', ['бан']),
    Synonyms('грамм', ['г', 'гр']),
    Synonyms('горсть'),
    Synonyms('зубчик'),
    Synonyms('килограмм', ['кг']),
    Synonyms('литр', ['л']),
    Synonyms('миллиграмм', ['мг']),
    Synonyms('миллилитр', ['мл']),
    Synonyms('пакетик'),
    Synonyms('полоска'),
    Synonyms('пучок', ['пуч']),
    Synonyms('сантиметр', ['см']),
    Synonyms('стакан', ['чашка', 'стак', 'ст']),
    Synonyms('стебель'),
    Synonyms('упаковка', ['уп', 'пачка']),
    Synonyms('штука', ['шт']),
    Synonyms('щепотка'),
]


names = []
mapping = {}
for record in NAMES:
    name = record.name
    names.append(name)
    mapping[name] = name
    for synonym in record.synonyms:
        names.append(synonym)
        mapping[synonym] = name


NAME = morph_pipeline(names).interpretation(
    Unit.type.normalized().custom(mapping.get)
)

NAME = rule(
    NAME,
    DOT.optional()
)

MODIFIER = morph_pipeline([
    'маленький',
    'большой',
    'неполный',
    'средний',
    'средне большой',
]).interpretation(
    Unit.modifier.normalized()
)

UNIT = rule(
    MODIFIER.optional(),
    or_(
        NAME,
        SPOON.interpretation(
            Unit.type
        )
    )
).interpretation(
    Unit
)


show_matches(
    UNIT,
    '1 стак.',
    '1/2 небольшой чайной ложки',
    'по 1 килограмму',
    'штука',
    'ложечку',
    'огромная упаковка'
)

1 стак.
  ─────


Unit(
    type='стакан',
    modifier=None
)

1/2 небольшой чайной ложки
              ────────────


Unit(
    type=Spoon(
        type='чайная'
    ),
    modifier=None
)

по 1 килограмму
     ──────────


Unit(
    type='килограмм',
    modifier=None
)

штука
─────


Unit(
    type='штука',
    modifier=None
)

ложечку
огромная упаковка
         ────────


Unit(
    type='упаковка',
    modifier=None
)

## Amount

### Number

In [16]:
def normalize_float(value):
    value = re.sub('[\s,.]+', '.', value)
    return float(value)


FLOAT = rule(
    INT,
    in_('.,'),
    INT
).interpretation(
    interp.custom(normalize_float)
)

DIGIT = INT.interpretation(
    interp.custom(int)
)


show_matches(
    FLOAT,
    '~ 3,14',
    '35.1$',
    '13'
)
show_matches(
    DIGIT,
    '15',
    '45.3'
)

~ 3,14
  ────


3.14

35.1$
──── 


35.1

13
15
──


15

45.3
── ─


[45, 3]

### Fraction

In [17]:
Fraction = fact(
    'Fraction',
    ['numerator', 'denominator']
)


NUMERATOR = INT.interpretation(
    Fraction.numerator.custom(int)
)

DENOMINATOR = INT.interpretation(
    Fraction.denominator.custom(int)
)

FRACTION = rule(
    NUMERATOR,
    in_('/\\'),
    DENOMINATOR
).interpretation(
    Fraction
)

UNICODES = {
    '½': Fraction(numerator=1, denominator=2),
    '⅓': Fraction(numerator=1, denominator=3),
    '⅔': Fraction(numerator=2, denominator=3),
    '¼': Fraction(numerator=1, denominator=4),
    '¾': Fraction(numerator=3, denominator=4)
}

UNICODE = in_(UNICODES).interpretation(
    interp.custom(UNICODES.get)
)


show_matches(
    FRACTION,
    r'1\2',
    '1/3'
)
show_matches(
    UNICODE,
    '½'
)

1\2
───


Fraction(
    numerator=1,
    denominator=2
)

1/3
───


Fraction(
    numerator=1,
    denominator=3
)

½
─


Fraction(
    numerator=1,
    denominator=2
)

### Literal

In [18]:
LITERALS = {
    'один': 1,
    'два': 2,
    'три': 3
}

LITERAL = dictionary(LITERALS).interpretation(
    interp.normalized().custom(LITERALS.get)
)


show_matches(
    LITERAL,
    'без двух',
    'один, два'
)

без двух
    ────


2

один, два
────  ───


[1, 2]

### Some

In [19]:
Some = fact('Some', [])


SOME = morph_pipeline([
    'по вкусу',
    'немного',
    'несколько',
]).interpretation(
    Some
)


show_matches(
    SOME,
    'немного'
)

немного
───────


Some()

### Range

In [20]:
Range = fact(
    'Range',
    ['start', 'stop']
)


VALUE = or_(
    DIGIT,
    UNICODE,
    FRACTION,
    FLOAT,
    LITERAL
)


RANGE = rule(
    VALUE.interpretation(Range.start),
    '-',
    VALUE.interpretation(Range.stop)
).interpretation(
    Range
)


show_matches(
    RANGE,
    '100-110',
    '1/3-1/2',
    
)

100-110
───────


Range(
    start=100,
    stop=110
)

1/3-1/2
───────


Range(
    start=Fraction(
        numerator=1,
        denominator=3
    ),
    stop=Fraction(
        numerator=1,
        denominator=2
    )
)

### Amount

In [21]:
Amount = fact(
    'Amount',
    ['value']
)


AMOUNT = or_(
    VALUE,
    RANGE,
    SOME
).interpretation(
    Amount.value
).interpretation(
    Amount
)


show_matches(
    AMOUNT,
    'один-два',
    '0,3'
)

один-два
────────


Amount(
    value=Range(
        start=1,
        stop=2
    )
)

0,3
───


Amount(
    value=0.3
)

## Measure

In [22]:
Measure = fact(
    'Measure',
    ['amount', 'unit']
)


MEASURE = rule(
    AMOUNT.interpretation(
        Measure.amount
    ),
    UNIT.interpretation(
        Measure.unit
    ).optional(),
).interpretation(
    Measure
)


show_matches(
    MEASURE,
    '2 неполных столовых ложки',
    '1 кг',
    '2-3 дольки'
)


2 неполных столовых ложки
─────────────────────────


Measure(
    amount=Amount(
        value=2
    ),
    unit=Unit(
        type=Spoon(
            type='столовая'
        ),
        modifier='неполный'
    )
)

1 кг
────


Measure(
    amount=Amount(
        value=1
    ),
    unit=Unit(
        type='килограмм',
        modifier=None
    )
)

2-3 дольки
───       


Measure(
    amount=Amount(
        value=Range(
            start=2,
            stop=3
        )
    ),
    unit=None
)

## Product

In [23]:
Product = fact(
    'Product',
    ['name', 'modifier']
)

### Name

In [24]:
names = []
with open('names.txt') as file:
    for line in file:
        name = line.rstrip()
        names.append(name)


seed(1)
sample(names, 10)

['ириска',
 'шелуха',
 'вишнёвка',
 'майоран',
 'закваска',
 'торт',
 'сок',
 'сушка',
 'пудра',
 'крахмал']

In [25]:
NAME = morph_pipeline(names).interpretation(
    Product.name.normalized()
)


show_matches(
    NAME,
    'Горчица',
    'табак',
    'солью'
)

Горчица
───────


'горчица'

табак
солью
─────


'соль'

### Modifier

In [26]:
Percent = fact(
    'Percent',
    ['value']
)
Adjs = fact(
    'Adjs',
    [attribute('parts').repeatable()]
)



PERCENT = rule(
    VALUE.interpretation(Percent.value),
    '%'
).interpretation(
    Percent
)

ADJ = or_(
    ADJF,
    PRTF,
).interpretation(
    interp.normalized()
).interpretation(
    Adjs.parts
)

ADJS = ADJ.repeatable(max=3).interpretation(
    Adjs
)

MODIFIER = or_(
    ADJS,
    PERCENT
).interpretation(
    Product.modifier
)


show_matches(
    MODIFIER,
    '15% сметана',
    'Сушеный розмарин',
    'сладкий красный перец'
)

15% сметана
───        


Percent(
    value=15
)

Сушеный розмарин
───────         


Adjs(
    parts=['сушёный']
)

сладкий красный перец
───────────────      


Adjs(
    parts=['сладкий',
     'красный']
)

### Product

In [27]:
PRODUCT = or_(
    rule(NAME),
    rule(NAME, MODIFIER),
    rule(MODIFIER, NAME)
).interpretation(
    Product
)


show_matches(
    PRODUCT,
    'картофель отварной',
    'сливочное масло',
    'творог',
    'неповторимый вкуснейший отварной болгарский перец'
)

картофель отварной
──────────────────


Product(
    name='картофель',
    modifier=Adjs(
        parts=['отварный']
    )
)

сливочное масло
───────────────


Product(
    name='масло',
    modifier=Adjs(
        parts=['сливочный']
    )
)

творог
──────


Product(
    name='творог',
    modifier=None
)

неповторимый вкуснейший отварной болгарский перец
             ────────────────────────────────────


Product(
    name='перец',
    modifier=Adjs(
        parts=['вкусный',
         'отварный',
         'болгарский']
    )
)

## Ingredient

In [28]:
Ingredient = fact(
    'Ingredient',
    ['measure', 'product']
)


_PRODUCT = PRODUCT.interpretation(Ingredient.product)

_MEASURE = MEASURE.interpretation(Ingredient.measure)

SEP = in_(':-')

INGREDIENT = or_(
    rule(
        _MEASURE,
        SEP.optional(),
        _PRODUCT
    ),
    rule(
        _PRODUCT,
        SEP.optional(),
        _MEASURE
    ),
    _PRODUCT
).interpretation(
    Ingredient
)


show_matches(
    INGREDIENT,
    '700г свинины',
    'скумбрия',
    'Мёд - 2 Чайных ложки',
    'Рис девзира 600 г',
    '2 средне больших моркови',
    'немного чернослива'
)

700г свинины
────────────


Ingredient(
    measure=Measure(
        amount=Amount(
            value=700
        ),
        unit=Unit(
            type='грамм',
            modifier=None
        )
    ),
    product=Product(
        name='свинина',
        modifier=None
    )
)

скумбрия
────────


Ingredient(
    measure=None,
    product=Product(
        name='скумбрия',
        modifier=None
    )
)

Мёд - 2 Чайных ложки
────────────────────


...

[Ingredient(
     measure=None,
     product=Product(
         name='рис',
         modifier=None
     )
 ),
 Ingredient(
     measure=Measure(
         amount=Amount(
             value=600
         ),
         unit=Unit(
             type='грамм',
             modifier=None
         )
     ),
     product=Product(
         name='девзира',
         modifier=None
     )
 )]

2 средне больших моркови
         ───────────────


Ingredient(
    measure=None,
    product=Product(
        name='морковь',
        modifier=Adjs(
            parts=['больший']
        )
    )
)

немного чернослива
──────────────────


Ingredient(
    measure=Measure(
        amount=Amount(
            value=Some()
        ),
        unit=None
    ),
    product=Product(
        name='чернослив',
        modifier=None
    )
)

# Extractor

In [30]:
from yargy.span import Span


class Line(Record):
    __attributes__ = ['offset', 'text']

    def __init__(self, offset, text):
        self.offset = offset
        self.text = text


class Match(Record):
    __attributes__ = ['span', 'fact']

    def __init__(self, span, fact):
        self.span = span
        self.fact = fact


class Result(Record):
    __attributes__ = ['text', 'matches']

    def __init__(self, text, matches):
        self.text = text
        self.matches = matches


def get_coverage(line, spans):
    cover = 0
    for start, stop in spans:
        cover += stop - start
    return cover / len(line)


def get_lines(text):
    for match in re.finditer('[^\n]+', text):
        yield Line(
            match.start(),
            match.group(0)
        )


def offset_span(span, offset):
    return Span(
        span.start + offset,
        span.stop + offset
    )


class Extractor(object):
    def __init__(self):
        self.parser = Parser(INGREDIENT)

    def process(self, line):
        matches = self.parser.findall(line.text)
        matches = sorted(matches, key=lambda _: _.span)
        spans = [_.span for _ in matches]
        coverage = get_coverage(line.text, spans)
        if coverage > 0.5:
            for match in matches:
                span = offset_span(match.span, line.offset)
                yield Match(span, match.fact)

    def __call__(self, text):
        text = preprocess(text)
        matches = []
        for line in get_lines(text):
            matches.extend(self.process(line))
        return Result(text, matches)

    
extractor = Extractor()
seed(12)
for text in sample(texts, 10):
    result = extractor(text)
    show_markup(result.text, [_.span for _ in result.matches])
    for match in result.matches:
        start, stop = match.span
        print(result.text[start:stop])
        display(match.fact)
    print('===' * 10)

Борщ украинский 
Ингредиенты: 
кости с мясом, 
─────   ─────  
свекла, 
──────  
картофель, 
─────────  
капуста свежая, 
──────────────  
лук, 
───  
морковь, 
───────  
мука (важно!), 
уксус (важно!), 
томатная паста (соус), 
──────────────  ────   
подсолнечное масло, 
──────────────────  
чеснок, 
──────  
перец, 
─────  
специи. 
──────  
Приготовление: 
1. Борщ одинаково вкусен и питателен как на мясном бульоне, так и на 
обычной воде. В первом случае отвариваем косточки с мясом полтора 
часа, не забываем снимать периодически пену с поверхности. 
2. Далее, в бульон или в обычную закипевшую воду бросаем картошку, 
нарезанную на средние кусочки, через минут 10 солим. Еще через 10 
минут бросаем нарезанную мелкой соломкой свежую капусту. Пока все это 
хозяйство варится, готовим поджарку: красную свеклу и морковку трем на
 крупной терке (а не режем соломкой, как обычно указывается!), 
высыпаем на сковороду с подогретым подсолнечном маслом, немного тушим,
 затем добавляем туда хорошег

Ingredient(
    measure=None,
    product=Product(
        name='кость',
        modifier=None
    )
)

мясом


Ingredient(
    measure=None,
    product=Product(
        name='мясо',
        modifier=None
    )
)

свекла


...

Ingredient(
    measure=Measure(
        amount=Amount(
            value=200
        ),
        unit=Unit(
            type='грамм',
            modifier=None
        )
    ),
    product=Product(
        name='пудра',
        modifier=Adjs(
            parts=['сахарный']
        )
    )
)

100 мл молока


Ingredient(
    measure=Measure(
        amount=Amount(
            value=100
        ),
        unit=Unit(
            type='миллилитр',
            modifier=None
        )
    ),
    product=Product(
        name='молоко',
        modifier=None
    )
)

1 пакетик ванилина


Ingredient(
    measure=Measure(
        amount=Amount(
            value=1
        ),
        unit=Unit(
            type='пакетик',
            modifier=None
        )
    ),
    product=Product(
        name='ванилин',
        modifier=None
    )
)